In [1]:
import numpy as np
import pandas as pd
import spacy
import nltk

In [2]:
df = pd.read_json("dataset//News_Category_Dataset_v3.json", lines=True)
print(df.head())

                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until you have a dog y

In [3]:
df = df.drop(columns=["link", "date", "authors"])

In [4]:
df

,headline,category,short_description
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha..."
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...
...,...,...,...
209522,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...
209523,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr..."
209524,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked..."
209525,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...


In [5]:
df.isna().sum()

headline             0
category             0
short_description    0
dtype: int64

In [6]:
print(df["headline"][0])
print(df.info())
print(df["category"].value_counts())
df["category"].count()

Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   headline           209527 non-null  object
 1   category           209527 non-null  object
 2   short_description  209527 non-null  object
dtypes: object(3)
memory usage: 4.8+ MB
None
category
POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS     

209527

In [7]:
df_limited = df.groupby('category').head(1000)


In [8]:
df_limited["category"].value_counts()
# df_limited["category"].count()

category
U.S. NEWS         1000
ARTS & CULTURE    1000
TRAVEL            1000
MONEY             1000
RELIGION          1000
LATINO VOICES     1000
IMPACT            1000
WEDDINGS          1000
COLLEGE           1000
PARENTS           1000
STYLE             1000
COMEDY            1000
GREEN             1000
TASTE             1000
HEALTHY LIVING    1000
THE WORLDPOST     1000
GOOD NEWS         1000
WORLDPOST         1000
FIFTY             1000
ARTS              1000
BLACK VOICES      1000
WOMEN             1000
HOME & LIVING     1000
QUEER VOICES      1000
PARENTING         1000
WORLD NEWS        1000
CULTURE & ARTS    1000
TECH              1000
SPORTS            1000
ENTERTAINMENT     1000
POLITICS          1000
WEIRD NEWS        1000
ENVIRONMENT       1000
EDUCATION         1000
CRIME             1000
SCIENCE           1000
WELLNESS          1000
BUSINESS          1000
STYLE & BEAUTY    1000
FOOD & DRINK      1000
MEDIA             1000
DIVORCE           1000
Name: count, dtype: int64

In [9]:
categories_to_remove = ["QUEER VOICES", "POLITICS", "IMPACT", "DIVORCE", "PARENTS", "WEIRD NEWS", "COLLEGE", \
        "LATINO VOICES", "CULTURE & ARTS", "BLACK VOICES", "GREEN", "TASTE", "WEDDINGS", "U.S. NEWS", "FIFTY", "ARTS" \
            "WOMEN",'SCIENCE', 'WELLNESS' \
, 'MEDIA', 'HOME & LIVING', 'WOMEN', \
 'MONEY', 'ARTS & CULTURE', 'STYLE', 'HEALTHY LIVING', \
 'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST', 'ARTS' \
    ,'COMEDY', 'PARENTING', 'WORLD NEWS', \
 'ENVIRONMENT', 'HEALTHY LIVING' \
    ]

df_limited = df_limited[~df_limited['category'].isin(categories_to_remove)]
df_limited["category"].value_counts()

unique_categories = df_limited['category'].unique()
print(unique_categories)

['TECH' 'SPORTS' 'ENTERTAINMENT' 'EDUCATION' 'CRIME' 'BUSINESS'
 'STYLE & BEAUTY' 'FOOD & DRINK' 'TRAVEL' 'RELIGION']


In [10]:
nlp = spacy.load('en_core_web_lg')


def cleaning_text(text):

    # print(text)
    text = text.lower()
    doc = nlp(text)

    new_line = []

    for w in doc :
        if(w.is_stop == False and w.is_punct == False and w.is_space == False):
            new_line.append(w.lemma_)
    # new_line = " ".join(new_line)

    # next = []
    # print(new_line)

    # for word in new_line :
    #     next.append(stemmer.stem(word))
    
    new_line = " ".join(new_line)

    return new_line

In [11]:
text = cleaning_text(df_limited["short_description"][13])
print(text)

man claim scamme people platform cause popular streamer consider twitch boycott


In [12]:
df_limited["short_description"] = df_limited['short_description'].apply(cleaning_text)

In [13]:
print(df_limited.head())

                                             headline       category  \
13  Twitch Bans Gambling Sites After Streamer Scam...           TECH   
17  Maury Wills, Base-Stealing Shortstop For Dodge...         SPORTS   
20  Golden Globes Returning To NBC In January Afte...  ENTERTAINMENT   
26  Las Vegas Aces Win First WNBA Title, Chelsea G...         SPORTS   
28  James Cameron Says He 'Clashed' With Studio Be...  ENTERTAINMENT   

                                    short_description  
13  man claim scamme people platform cause popular...  
17  maury will help los angeles dodgers win world ...  
20  past 18 month hollywood effectively boycott gl...  
26       las vegas professional sport champion sunday  
28  avatar director say aspect 2009 movie competit...  


In [14]:
df_limited["short_description"][13]

'man claim scamme people platform cause popular streamer consider twitch boycott'

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout


In [16]:
model = Sequential()

print(df_limited["short_description"])


13        man claim scamme people platform cause popular...
17        maury will help los angeles dodgers win world ...
20        past 18 month hollywood effectively boycott gl...
26             las vegas professional sport champion sunday
28        avatar director say aspect 2009 movie competit...
                                ...                        
142424       sure star recently golden globe damn look good
142425    aubrey plaza tina fey star wear sandra bullock...
142433    want know gorgeous pregnant mermaid look like ...
142440    want sure check huffpost style twitter faceboo...
142441    emma stone noticeably absent 2014 golden globe...
Name: short_description, Length: 10000, dtype: object


In [17]:
tokenizer = Tokenizer(num_words=10000)

tokenizer.fit_on_texts(df_limited["short_description"])

description = tokenizer.texts_to_sequences(df_limited["short_description"])

maxi = 0
length = len(description)


for i in range(0, length):
    if len(description[i]) > maxi: 
        maxi = len(description[i])

print(maxi)


35


In [18]:
description

[[35, 199, 5132, 7, 704, 350, 489, 7327, 404, 7328, 3324],
 [7329, 405, 24, 449, 490, 3325, 85, 16, 200, 1081, 303, 973, 5133, 168],
 [192,
  613,
  61,
  884,
  5134,
  3324,
  1152,
  83,
  7330,
  5135,
  271,
  505,
  68,
  1457,
  116,
  112,
  271],
 [1354, 1153, 470, 322, 845, 226],
 [7331, 435, 1, 1288, 2507, 376, 2243, 8],
 [435, 1082, 5136, 227, 670, 1022, 585, 2508, 3, 3326, 3327, 2509, 2510],
 [7332,
  4024,
  2019,
  48,
  156,
  1567,
  169,
  1154,
  5137,
  129,
  94,
  222,
  7333,
  1083,
  7334,
  3,
  64],
 [450, 559, 1, 28, 193, 8, 436, 3, 364, 376],
 [451, 17, 36, 544, 2511, 4025, 1458, 28],
 [1354, 1153, 3328, 54, 20, 151, 4026, 927, 1459, 2, 258],
 [1460, 85, 642, 614, 17, 731, 32, 2244, 7335, 6, 3329, 5138, 1355],
 [1697, 170, 732, 323, 2245, 107, 249, 103, 1568, 1698, 2020, 286],
 [43,
  39,
  1084,
  2512,
  1569,
  324,
  199,
  83,
  491,
  492,
  1849,
  104,
  1850,
  1851,
  615,
  109,
  471,
  7336,
  3330],
 [435, 55, 9, 7337, 376, 7338],
 [705, 2246,

In [19]:
description = pad_sequences(description, padding='post', maxlen=maxi)

In [20]:
description.shape

(10000, 35)

In [76]:
from tensorflow.keras.layers import Flatten

model = Sequential()
model.add(Embedding(input_dim= 10000, output_dim= 64))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Flatten())
model.add(Dense(1, activation='softmax'))
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True) , metrics=['accuracy'])

c:\Users\Apurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [77]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_limited["category"] = le.fit_transform(df_limited["category"])

print(type(df_limited["category"]))
print(type(description))

model.summary()

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_20 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_21 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [78]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(description, df_limited["category"], test_size=0.2, random_state=42)


In [79]:
y_train.shape

(8000,)

In [80]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10


c:\Users\Apurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:827: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\Users\Apurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(32, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)
c:\Users\Apurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\backend\tensorflow\nn.py:636: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not repres

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.0963 - loss: 0.0000e+00

c:\Users\Apurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:827: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\Users\Apurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 43ms/step - accuracy: 0.0963 - loss: 0.0000e+00 - val_accuracy: 0.1100 - val_loss: 0.0000e+00
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.0999 - loss: 0.0000e+00 - val_accuracy: 0.1100 - val_loss: 0.0000e+00
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.1016 - loss: 0.0000e+00 - val_accuracy: 0.1100 - val_loss: 0.0000e+00
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.1012 - loss: 0.0000e+00 - val_accuracy: 0.1100 - val_loss: 0.0000e+00
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.1022 - loss: 0.0000e+00 - val_accuracy: 0.1100 - val_loss: 0.0000e+00
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.0986 - loss: 0.0000e+00 - val_accuracy: 0.1100 - val_loss: 0.0000e+00
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.0954 - loss: 0.0000e+00 - val_accuracy: 0.1100 - val_loss: 0.0000e+00
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms

In [85]:
model.evaluate(X_test, y_test)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.1039 - loss: 0.0000e+00


[0.0, 0.10999999940395355]